In [1]:
using LinearAlgebra
using Dates

using Plots
using BenchmarkTools

BLAS.set_num_threads(1)

include("src/io.jl")
include("src/distances.jl")
include("src/optimizer.jl")
include("src/network.jl")
include("src/base.jl")
include("src/pretraining.jl")

# Initialize the parameters
globalParms, MCParms, NNParms, preTrainParms, systemParmsList = parametersInit()

# Initialize the input data
inputs = inputInit(globalParms, NNParms, preTrainParms, systemParmsList)
if globalParms.mode == "training"
    model, opt, refRDFs = inputs
else
    model = inputs
end

Running ML-IMC in the training mode.
Building a model...
Chain(Dense(20 => 20, relu; bias=false), Dense(20 => 20, relu; bias=false), Dense(20 => 1; bias=false))
   Number of layers: 4 
   Number of neurons in each layer: [20, 20, 20, 1]


(Chain(Dense(20 => 20, relu; bias=false), Dense(20 => 20, relu; bias=false), Dense(20 => 1; bias=false)), AMSGrad(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}()), Any[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.004, 1.005, 1.006, 1.005, 1.006, 1.006, 1.007, 1.007, 1.007, 1.007], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.003, 1.003, 1.003, 1.004, 1.004, 1.004, 1.005, 1.005, 1.004, 1.005], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.004, 1.004, 1.004, 1.005, 1.005, 1.005, 1.005, 1.005, 1.005, 1.005], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.004, 1.004, 1.005, 1.004, 1.006, 1.003, 1.004, 1.004, 1.005, 1.004]])

In [2]:
systemParms = systemParmsList[1]

systemParameters("100CH3OH-CG", "methanol-data/100CH3OH/100CH3OH-CG-200.xtc", "methanol-data/100CH3OH/100CH3OH-CG.pdb", 512, "MET", [32.433, 32.433, 32.433], 34116.256126737, "methanol-data/100CH3OH/100CH3OH-CG.rdf", 300, 0.05, 0.0, 298.15, 0.40339559569659034, 1.5, 0.5)

In [3]:
traj = readXTC(systemParms)
frame = read_step(traj, 1)
coordinates = positions(frame)

3×512 Chemfiles.ChemfilesArray:
 18.08  15.83  13.55  14.73  12.49  …  17.62   4.33  24.61   8.32   0.52
  4.76  32.47  12.07  18.75  32.29     14.16  -0.4   11.53  26.35  23.4
 27.28  31.76  13.96   4.45   3.85      3.67  24.95  29.58  21.27  29.59

In [4]:
distanceMatrix = buildDistanceMatrix(frame)

512×512 Matrix{Float64}:
  0.0       7.09189  15.855    17.4607   …   9.68317  15.981    20.8271
  7.09189   0.0      19.4916   14.7913      14.8578   14.2792   17.9268
 15.855    19.4916    0.0      11.6814      19.1468   16.8733   23.2905
 17.4607   14.7913   11.6814    0.0         14.4029   18.7578   16.7659
 12.0278    5.86375  16.1165   13.7371      18.4128   16.9396   16.4657
  6.82342   7.67516  16.2445   13.3721   …  12.6422   17.2571   20.5049
 13.0989   18.7139   17.984    24.9762      11.0264   17.6202   18.7331
 11.3263   10.5723   11.0429   10.3943      14.228    21.6119   20.0425
 19.6562   18.8416   18.4182   13.9998      13.8353   11.5905    6.22577
 15.2346   17.5955   23.1113   24.493       16.9502   11.3124   10.3767
  9.03241  12.3969   11.9791   16.8481   …  15.5818   15.1848   18.55
 15.0417   17.157    15.4381   15.8653      13.3199   15.7629   12.5644
 15.2687   12.0559    9.79153  15.5865      16.0364   18.6892   24.1867
  ⋮                                     

It seems that one can change eta parameters instead of Rs!
Rs can stay zero or be shifted a little bit (for H2O in n2p2 Rs is from 0 to ~1 Å)

eta parameter on the other hand, goes from 0 to 1.5 Bohr^-2 in the same example:
[0.001; 0.01, 0.03, 0.06, 0.15, 0.30, 0.60, 1.50 Bohr^-2]



Converting to sigmas Å:

In [37]:
map(x -> sqrt(1 / (2 * x)), [0.001, 0.01, 0.03, 0.06, 0.15, 0.30, 0.60, 1.50]) .* 0.5292

8-element Vector{Float64}:
 11.833271736928888
  3.7420090860392095
  2.1604499531347634
  1.52766881227575
  0.9661825914391131
  0.6831942622709883
  0.48309129571955656
  0.30553376245514996

Converting to etas Å$^{-2}$:

In [39]:
map(x -> 0.5292^-2 * x, [0.001, 0.01, 0.03, 0.06, 0.15, 0.30, 0.60, 1.50])

8-element Vector{Float64}:
 0.0035707572690619875
 0.035707572690619874
 0.10712271807185962
 0.21424543614371924
 0.5356135903592981
 1.0712271807185962
 2.1424543614371925
 5.356135903592981

The lambda should be +1 or -1 (shifts the maxima of cosine to 0 or pi radians)


etas for angular symmetry functions are usually less than zero, from 1E-3 to 1E-1:

Converting to Å$^{-2}$:

In [43]:
map(x -> 0.5292^-2 * x, [0.001, 0.01, 0.03, 0.07])

4-element Vector{Float64}:
 0.0035707572690619875
 0.035707572690619874
 0.10712271807185962
 0.24995300883433913


zetas are set to 1-6 (high zeta gives a narrower range of nonzero values)

Now I'm going to compute the narrow angular symmetry function for a single atom with fixed parameters.

In [46]:
rmax = 6 # Å
rs = 0 # Å 
eta = 0.1 # Å^-2
lambda = 1
zeta = 1;

In [48]:
distanceVector = distanceMatrix[:, 1];

I'm thinking about using sparse arrays for saving indeces of non-zero values of G3:

In [96]:
using SparseArrays

In [113]:
N = length(distanceVector)

512

In [151]:
function buildNeighborMatrix1(distanceVector, rmax)
    N = length(distanceVector)
    withinCutoff = Matrix{Bool}(zeros(N, N))
    for k in eachindex(distanceVector)
        for j in 1:k-1
            withinCutoff[j, k] = distanceVector[j] < rmax && distanceVector[k] < rmax 
        end
    end
    return (withinCutoff)
end

buildNeighborMatrix1 (generic function with 1 method)

In [154]:
function buildNeighborMatrix2(distanceVector, rmax)
    N = length(distanceVector)
    withinCutoff = SparseMatrixCSC{Bool, Int64}(spzeros(N, N));
    for k in eachindex(distanceVector)
        for j in 1:k-1
            withinCutoff[j, k] = distanceVector[j] < rmax && distanceVector[k] < rmax 
        end
    end
    return (withinCutoff)
end

buildNeighborMatrix2 (generic function with 1 method)

In [140]:
withinCutoffDefault = buildNeighborMatrix1(distanceVector, rmax);

In [142]:
withinCutoffSparse = buildNeighborMatrix2(distanceVector, rmax);

In [145]:
Matrix{Bool}(withinCutoffSparse) == withinCutoffDefault

true

In [150]:
@btime buildNeighborMatrix1($distanceVector, $rmax);

  367.438 μs (4 allocations: 2.25 MiB)


In [155]:
@btime buildNeighborMatrix2($distanceVector, $rmax);

  654.971 μs (12 allocations: 10.77 KiB)


Building a sparse matrix takes longer time, but requires much less memory!

In [159]:
@btime sum($withinCutoffDefault);

  2.051 μs (0 allocations: 0 bytes)


In [161]:
@btime sum($withinCutoffSparse);

  357.769 μs (0 allocations: 0 bytes)


It is much slower to access the elements in the sparse matrices! Maybe I should use the normal ones...

In [163]:
@btime Matrix{Bool}(zeros(N, N));

  190.539 μs (4 allocations: 2.25 MiB)


Half of the time is needed to allocate the matrix! If the matrix is passed to the function, then it should be twice as fast!

In [297]:
function updateNeighborMatrix!(withinCutoff, distanceVector, rmax)
    for k in eachindex(distanceVector)
        for j in 1:k-1
            withinCutoff[j, k] = 0 < distanceVector[j] < rmax && 0 < distanceVector[k] < rmax
        end
    end
    return (withinCutoff)
end

updateNeighborMatrix! (generic function with 1 method)

In [298]:
withinCutoff = Matrix{Bool}(zeros(N, N));

In [299]:
withinCutoff = updateNeighborMatrix!(withinCutoff, distanceVector, rmax);

In [300]:
@btime updateNeighborMatrix!($withinCutoff, $distanceVector, $rmax);

  84.465 μs (0 allocations: 0 bytes)


In [301]:
sum(withinCutoff)

78

In [203]:
@btime distance(frame, 0, 1)

  15.935 ns (0 allocations: 0 bytes)


7.091892808138942

In [259]:
vector_ij = positions(frame)[:, 2] .- positions(frame)[:, 1];
vector_ik = positions(frame)[:, 3] .- positions(frame)[:, 1];

In [260]:
@btime dot($vector_ij, $vector_ik)

  6.202 ns (0 allocations: 0 bytes)


153.0790194234857

In [261]:
@btime reduce(+, $vector_ij .* $vector_ik)

  20.077 ns (1 allocation: 80 bytes)


153.0790194234857

In [364]:
function getTripletGeometry(frame, i, j, k)
    @assert i != j
    @assert i != k
    @assert k != j
    vector_ij = positions(frame)[:, j] .- positions(frame)[:, i]
    vector_ik = positions(frame)[:, k] .- positions(frame)[:, i]
    # Chemfiles has 0-based arrays
    distance_ij = distance(frame, i - 1, j - 1)
    distance_ik = distance(frame, i - 1, k - 1)
    distance_kj = distance(frame, k - 1, j - 1)
    cosAngle = dot(vector_ij, vector_ik) / (distance_ij * distance_ik)
    return (cosAngle, distance_ij, distance_ik, distance_kj)
end

getTripletGeometry (generic function with 1 method)

In [302]:
findall(x -> x != 0, withinCutoff)

78-element Vector{CartesianIndex{2}}:
 CartesianIndex(14, 27)
 CartesianIndex(14, 129)
 CartesianIndex(27, 129)
 CartesianIndex(14, 240)
 CartesianIndex(27, 240)
 CartesianIndex(129, 240)
 CartesianIndex(14, 284)
 CartesianIndex(27, 284)
 CartesianIndex(129, 284)
 CartesianIndex(240, 284)
 CartesianIndex(14, 285)
 CartesianIndex(27, 285)
 CartesianIndex(129, 285)
 ⋮
 CartesianIndex(14, 483)
 CartesianIndex(27, 483)
 CartesianIndex(129, 483)
 CartesianIndex(240, 483)
 CartesianIndex(284, 483)
 CartesianIndex(285, 483)
 CartesianIndex(286, 483)
 CartesianIndex(328, 483)
 CartesianIndex(416, 483)
 CartesianIndex(427, 483)
 CartesianIndex(443, 483)
 CartesianIndex(461, 483)

In [361]:
@btime getTripletGeometry($frame, 1, 461, 483)

  287.294 ns (14 allocations: 800 bytes)


(0.7169325422695667, 3.4314144165171827, 5.791044424571701)

In [363]:
@btime getTripletGeometry($frame, 1, 461, 483)

  299.824 ns (14 allocations: 800 bytes)


(0.7169325422695667, 3.4314144165171827, 5.791044424571701, 4.100949724873194)

In [329]:
getTripletGeometry(frame, 1, 461, 483)

(0.7169325422695667, 3.4314144165171827, 5.791044424571701)

In [372]:
function G9(cosAngle, distance_ij, distance_ik, cutoff, eta, zeta, lambda=1, rshift=0)
    return (
        (1 + lambda * cosAngle)^zeta * 
        exp(-eta * (
                (distance_ij - rshift)^2 + 
                (distance_ik - rshift)^2)
                ) * 
        distanceCutoff(distance_ij, cutoff) * 
        distanceCutoff(distance_ik, cutoff))
end        

G9 (generic function with 3 methods)

In [373]:
function G3(cosAngle, distance_ij, distance_ik, distance_kj, cutoff, eta, zeta, lambda=1, rshift=0)
    return (
        (1 + lambda * cosAngle)^zeta * 
        exp(-eta * (
                (distance_ij - rshift)^2 + 
                (distance_ik - rshift)^2 +
                (distance_kj - rshift)^2)) * 
        distanceCutoff(distance_ij, cutoff) * 
        distanceCutoff(distance_ik, cutoff) *
        distanceCutoff(distance_kj, cutoff))
end        

G3 (generic function with 3 methods)

In [402]:
function G9total(i, frame, distanceMatrix, cutoff, eta, zeta, lambda=1.0, rshift=0.0)
    sum = 0.0
    # i is the index of the central atom
    distanceVector = distanceMatrix[:, i];
    N = length(distanceVector)

    # Get indexes of valid j and k atoms
    withinCutoff = Matrix{Bool}(zeros(N, N));
    withinCutoff = updateNeighborMatrix!(withinCutoff, distanceVector, cutoff)
    validPairs = findall(x -> x != 0, withinCutoff)
    
    for indexes in validPairs
        j, k = Tuple(indexes)
        cosAngle, distance_ij, distance_ik, distance_kj = getTripletGeometry(frame, i, j, k)
        sum += G9(cosAngle, distance_ij, distance_ik, cutoff, eta, zeta, lambda, rshift)
    end
    return (2.0^(1.0-zeta) * sum)
end

G9total (generic function with 3 methods)

In [403]:
function G3total(i, frame, distanceMatrix, cutoff, eta, zeta, lambda=1.0, rshift=0.0)
    sum = 0.0
    # i is the index of the central atom
    distanceVector = distanceMatrix[:, i];
    N = length(distanceVector)

    # Get indexes of valid j and k atoms
    withinCutoff = Matrix{Bool}(zeros(N, N));
    withinCutoff = updateNeighborMatrix!(withinCutoff, distanceVector, cutoff)
    validPairs = findall(x -> x != 0, withinCutoff)
    
    for indexes in validPairs
        j, k = Tuple(indexes)
        cosAngle, distance_ij, distance_ik, distance_kj = getTripletGeometry(frame, i, j, k)
        sum += G3(cosAngle, distance_ij, distance_ik, distance_kj, cutoff, eta, zeta, lambda, rshift)
    end
    return (2.0^(1.0-zeta) * sum)
end

G3total (generic function with 3 methods)

In [376]:
cutoff = 6 # Å
rs = 0 # Å 
eta = 0.1 # Å^-2
lambda = 1
zeta = 1;

In [409]:
G9total(1, frame, distanceMatrix, cutoff, eta, 1, 1)

0.028941013108324705

In [410]:
@btime G9total(1, $frame, $distanceMatrix, 4, eta, zeta, lambda)

  307.902 μs (28 allocations: 2.29 MiB)


0.00017162994385048377

In [370]:
@btime G9total(1, $frame, $distanceMatrix, 6, eta, zeta, lambda)

  331.377 μs (1106 allocations: 2.35 MiB)


0.028941013108324705

In [371]:
@btime G9total(1, $frame, $distanceMatrix, 8, eta, zeta, lambda)

  471.404 μs (6524 allocations: 2.65 MiB)


0.15068459491431355

In [379]:
G3total(1, frame, distanceMatrix, cutoff, eta, zeta, lambda)

0.0006120032153743085

In [380]:
@btime G3total(1, $frame, $distanceMatrix, 4, eta, zeta, lambda)

  306.303 μs (28 allocations: 2.29 MiB)


0.0

In [381]:
@btime G3total(1, $frame, $distanceMatrix, 6, eta, zeta, lambda)

  334.992 μs (1106 allocations: 2.35 MiB)


0.0006120032153743085

In [382]:
@btime G3total(1, $frame, $distanceMatrix, 8, eta, zeta, lambda)

  492.737 μs (6524 allocations: 2.65 MiB)


0.006419453311174974

In [390]:
@btime G2total($distanceVector, 6, 0, 2)

  2.492 μs (0 allocations: 0 bytes)


1.2261216304304188